In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
from langchain.sql_database import SQLDatabase
import warnings
import re
import os

warnings.filterwarnings("ignore")

c:\Users\Swapnil Singh\Desktop\Projects\Gen AI project\DBChatPro\dbchatpro\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define your database connection parameters
db_user = "root"
db_password = "1256"
db_host = "localhost"
db_name = "test_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

# %%
print(db.get_usable_table_names())



['category', 'catregory_icon', 'customer', 'customer_feedbacks', 'delivery', 'inventory', 'marketing', 'order_item', 'orders', 'products', 'rating', 'test']


In [3]:
# %%
db_schema = db.get_table_info()



In [4]:
# %%
# Load environment variables
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# %%
model = genai.GenerativeModel('gemini-pro')



In [5]:
# %%
question = "What many products name are there in products table?"

# %%
response = model.generate_content(question)

# %%
response.text



'```sql\nSELECT COUNT(DISTINCT name) AS num_product_names\nFROM products;\n```'

In [6]:
# %%
few_shots = [
    {
        "input": "How many Distinct Products are present?",
        "query": "SELECT COUNT(DISTINCT product_id) FROM products;"
    },
    {
        "input": "What is the total number of orders placed?",
        "query": "SELECT COUNT(*) FROM orders;"
    },
    {
        "input": "Which product has the highest number of orders?",
        "query": "SELECT product_id, COUNT(*) AS order_count FROM order_item GROUP BY product_id ORDER BY order_count DESC LIMIT 1;"
    },
    {
        "input": "How many customers have placed at least one order?",
        "query": "SELECT COUNT(DISTINCT customer_id) FROM orders;"
    },
    {
        "input": "Which customer has placed the highest number of orders?",
        "query": "SELECT customer_id, COUNT(*) AS order_count FROM orders GROUP BY customer_id ORDER BY order_count DESC LIMIT 1;"
    },
    {
        "input": "Which category has the highest number of products?",
        "query": "SELECT category_id, COUNT(*) AS product_count FROM products GROUP BY category_id ORDER BY product_count DESC LIMIT 1;"
    },
    {
        "input": "Retrieve customer feedback along with their order details.",
        "query": "SELECT c.customer_id, c.customer_name, cf.feedback_id, o.order_id FROM customer c JOIN customer_feedbacks cf ON c.customer_id = cf.customer_id JOIN orders o ON c.customer_id = o.customer_id;"
    },
    {
        "input": "List top 5 customer names who ordered 3 times.",
        "query": "SELECT c.customer_name, COUNT(o.order_id) AS order_count FROM customer c JOIN orders o ON c.customer_id = o.customer_id GROUP BY c.customer_id, c.customer_name HAVING order_count = 3 ORDER BY c.customer_name LIMIT 5;"
    }   
]


# %%
prompt = [
        f"""
            You are an expert in converting English questions to SQL query!
            The SQL database has many tables, and these are the schemas: {db_schema}, 
            You can order the results by a relevant column to return the most interesting examples in the database,
            Never query for all the columns from a specific table, only ask for the relevant columns given the question.

            Important: Also the sql code should not have ``` in beginning or end and sql word in output. Please check the column names before executing any query 
            
            You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

            DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

            If the question does not seem related to the database, just return "I don't know" as the answer.

            Here are some examples of user inputs and their corresponding SQL queries:
            """,
    ]

# %%
# Append each example to the prompt
for sql_example in few_shots:
    prompt.append(
        f"\nExample - {sql_example['input']}, the SQL command will be something like this {sql_example['query']}")

# Join prompt sections into a single string
formatted_prompt = [''.join(prompt)]



In [7]:
# %%
question2 = "List top 2 customer names who ordered 3 times"

# %%
response_with_few_shots = model.generate_content([formatted_prompt[0], question2])

# %%
response_with_few_shots = re.sub(r"(\n|```sql|```|sql)", " ", response_with_few_shots.text).strip()

# %%
response_with_few_shots




'SELECT c.customer_name, COUNT(o.order_id) AS order_count  FROM customer c  JOIN orders o ON c.customer_id = o.customer_id  GROUP BY c.customer_id, c.customer_name  HAVING order_count = 3  ORDER BY c.customer_name  LIMIT 2;'

In [ ]:
# %%
sql_response = db.run(response_with_few_shots)

# %%
response = model.generate_content(f"""
Based on the sql response, Give answer based on given dataset {db_schema} and related to the {question2} asked:

SQL response: {sql_response}

""")

print(response.text)

1. Aahana Menon - 3 orders
2. Aarav Rout - 3 orders
